<a href="https://colab.research.google.com/github/Amplil/laser/blob/master/laser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [LASER for NLP tasks - Part 2](https://www.engati.com/blog/laser-for-nlp-tasks-part-ii)のページを参考にしたプログラム

In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/packages")

In [ ]:
#!git clone https://github.com/facebookresearch/LASER

In [ ]:
cd "/content/drive/My Drive/Colab Notebooks/LASER_official"

/content/drive/My Drive/Colab Notebooks/LASER_official


In [ ]:
ls

CODE_OF_CONDUCT.md  install_external_tools.sh  remove_external_tools.sh
CONTRIBUTING.md     install_models.sh          source/
data/               LICENSE                    tasks/
docker/             README.md


In [ ]:
!export LASER="/content/drive/My Drive/Colab Notebooks/LASER"

In [ ]:
!export LASER="${HOME}/projects/laser"

In [ ]:
!bash ./install_models.sh

Please set the environment variable 'LASER'


In [ ]:
# !pip install --target "/content/drive/MyDrive/Colab Notebooks/packages" laserembeddings

In [ ]:
# cd "/content/drive/MyDrive/Colab Notebooks/packages"

In [ ]:
!pip install laserembeddings

     |████████████████████████████████| 51kB 2.9MB/s 
     |████████████████████████████████| 860kB 6.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp37-none-any.whl size=883999 sha256=af684a18c1283ce0102b03ffb24b68a824e4ad30f538c761294bd24280c31fd8
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [ ]:
!python -m laserembeddings download-models


✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


In [ ]:
!python -m laserembeddings download-test-data

❌  Looks like you're not running laserembeddings from its source code
     → please checkout https://github.com/yannvgn/laserembeddings.git
       then run "python -m laserembeddings download-test-data" from the root of the repository


In [ ]:
!pytest tests/test_laser.py

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1
rootdir: /content, inifile:

========================= no tests ran in 0.00 seconds =========================
ERROR: file not found: tests/test_laser.py



In [ ]:
!pip install laserembeddings[ja]

     |████████████████████████████████| 491kB 5.8MB/s 
     |████████████████████████████████| 13.4MB 40.9MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-cp37-none-any.whl size=13556725 sha256=bb9f9120835e9667584a6d92360e9a1aca2217e08a5764ecd335ecff7fcd9a82
  Stored in directory: /root/.cache/pip/wheels/ff/00/d1/0c094a0ce58a77199a0c5801f0ecf510c80f0ecbec27f07d2c
Successfully built ipadic


## kaggleのコンペTweet Sentiment Extractionのtest.csvを使う
https://www.kaggle.com/c/tweet-sentiment-extraction/data?select=test.csv

In [ ]:
#df_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/tweet_train.csv', index_col=0).fillna('')
#df_train

,keyword,location,text,target
id,,,,
1,,,Our Deeds are the Reason of this #earthquake M...,1
4,,,Forest fire near La Ronge Sask. Canada,1
5,,,All residents asked to 'shelter in place' are ...,1
6,,,"13,000 people receive #wildfires evacuation or...",1
7,,,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...
10869,,,Two giant cranes holding a bridge collapse int...,1
10870,,,@aria_ahrary @TheTawniest The out of control w...,1
10871,,,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1


In [ ]:
df_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/tweet-sentiment-test.csv', index_col=0).fillna('')
df_train

,text,sentiment
textID,,
f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
96d74cb729,Shanghai is also really exciting (precisely -...,positive
eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
01082688c6,happy bday!,positive
33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive
...,...,...
e5f0e6ef4b,"its at 3 am, im very tired but i can`t sleep ...",negative
416863ce47,All alone in this old house again. Thanks for...,positive
6332da480c,I know what you mean. My little dog is sinkin...,negative


In [ ]:
#特徴データ
feature = df_train.text
#ラベルデータ
target = df_train.sentiment

In [ ]:
#scikit-learnを用いたデータ分割
from sklearn.model_selection import train_test_split
X1, X2, Y1, Y2 = train_test_split(feature, target, test_size=0.3)

In [ ]:
X1

textID
4975dbda48           i wish i wasn`t all  bummed but i kinda am
40d5393a6e    Signing off for the night. Watching 'The Reade...
1c2a41d71e     At work at the mo and it`s so sunny here toda...
ee35fcd036    this evening,picking up n`pop and go for dinne...
ca333e378d    I`m really bored! My brother went to get his p...
                                    ...                        
5026e37179    Kinda doing nothing. Life is boring. I feel li...
b8332c63c9     If the drive is automatically ejecting suppor...
83e576fd29               Listening to Busted - 3AM. I miss them
20f35978fb     I`m up wit cha!! Just got home from da reggae...
981a6f186d    The music video I made for my  project for `Th...
Name: text, Length: 2473, dtype: object

In [ ]:
Y1

textID
4975dbda48    positive
40d5393a6e     neutral
1c2a41d71e     neutral
ee35fcd036     neutral
ca333e378d    negative
                ...   
5026e37179    negative
b8332c63c9     neutral
83e576fd29    negative
20f35978fb    positive
981a6f186d     neutral
Name: sentiment, Length: 2473, dtype: object

In [ ]:
from laserembeddings import Laser
laser=Laser()
embeddings = laser.embed_sentences(
    ['let your neural network be polyglot',
     'use multilingual embeddings!'],
    lang='en')  # lang is only used for tokenization
embeddings

array([[0.00156426, 0.01067378, 0.00388247, ..., 0.021797  , 0.0062856 ,
        0.01434554],
       [0.01456157, 0.00154333, 0.00090701, ..., 0.01484861, 0.00712931,
        0.0476219 ]], dtype=float32)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.utils import to_categorical
import re
import pandas as pd

In [ ]:
input_dict = {
    "en":"Machine learning isn't as hard as people think.",
    "de":"Maschinelles Lernen ist nicht so schwer wie die Leute denken.",
    "fr":"L'apprentissage automatique n'est pas aussi difficile qu'on le pense.",
    "it":"L'apprendimento automatico non è così difficile come la gente pensa.",
    "es":"El aprendizaje automático no es tan difícil como la gente piensa.",
    "ru":"Машинное обучение не так сложно, как думают люди.",
    "ar":"التعلم الآلي ليس صعبا كما يظن الناس.",
    "pt":"O aprendizado de máquina não é tão difícil quanto as pessoas pensam.",
}

In [ ]:
model=Sequential()
model.add(Dense(3,activation='softmax'))
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(X1,Y1,validation_data=(X2,Y2),epochs=5)

Epoch 1/5


ValueError: ignored

In [ ]:
embeddings = laser.embed_sentences(input_dict,
    lang='en')  # lang is only used for tokenization

In [ ]:
embeddings

In [ ]:
embedded_dict = {}
for key in input_dict:
    embedded_dict[key] = np.array(get_vect(input_dict[key], lang = key))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cos_lib = cosine_similarity(embedded_dict['it'], embedded_dict['es'])

In [ ]:
embeddings[0].shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cos_lib = cosine_similarity(embeddings[0], embeddings[1])

In [ ]:
embeddings[0]

# laser embeddings

In [ ]:
# import fasttext
import ftfy
import html
import laserembeddings
import numpy as np
import pandas as pd
import re
import tensorflow as tf
import sys

from fastcache import clru_cache
from laserembeddings import Laser
from typing import List, Union
from urllib.parse import unquote
from sklearn.model_selection import train_test_split

ModuleNotFoundError: ignored

# [kaggleのページ](https://www.kaggle.com/jamesmcguigan/nlp-laser-embeddings-keras)を参考にしたプログラム（直接kaggleの方でもプログラムを実行できる）

In [ ]:
%%bash
# DOCS: https://github.com/facebookresearch/LASER/blob/master/install_models.sh

mkdir -p models/laser/
# for FILE in bilstm.eparl21.2018-11-19.pt eparl21.fcodes eparl21.fvocab bilstm.93langs.2018-12-26.pt 93langs.fcodes 93langs.fvocab; do
for FILE in bilstm.93langs.2018-12-26.pt 93langs.fcodes 93langs.fvocab; do
    wget -cq https://dl.fbaipublicfiles.com/laser/models/$FILE -O models/laser/$FILE
done

In [ ]:
#!pip install --target "/content/drive/MyDrive/Colab Notebooks/packages" ftfy

In [ ]:
#!pip install --target "/content/drive/MyDrive/Colab Notebooks/packages" fastcache

In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/packages")

In [ ]:
# import fasttext
import ftfy
import html
import laserembeddings
import numpy as np
import pandas as pd
import re
import tensorflow as tf
import sys

from fastcache import clru_cache
from laserembeddings import Laser
from typing import List, Union
from urllib.parse import unquote
from sklearn.model_selection import train_test_split

ModuleNotFoundError: ignored